In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hdramy-music-dataset/94_180.wav
/kaggle/input/hdramy-music-dataset/121_420.wav
/kaggle/input/hdramy-music-dataset/92_030.wav
/kaggle/input/hdramy-music-dataset/17_570.wav
/kaggle/input/hdramy-music-dataset/27_1080.wav
/kaggle/input/hdramy-music-dataset/29_390.wav
/kaggle/input/hdramy-music-dataset/62_510.wav
/kaggle/input/hdramy-music-dataset/28_270.wav
/kaggle/input/hdramy-music-dataset/125_000.wav
/kaggle/input/hdramy-music-dataset/25_060.wav
/kaggle/input/hdramy-music-dataset/44_270.wav
/kaggle/input/hdramy-music-dataset/114_270.wav
/kaggle/input/hdramy-music-dataset/50_390.wav
/kaggle/input/hdramy-music-dataset/113_150.wav
/kaggle/input/hdramy-music-dataset/111_000.wav
/kaggle/input/hdramy-music-dataset/101_270.wav
/kaggle/input/hdramy-music-dataset/89_090.wav
/kaggle/input/hdramy-music-dataset/116_330.wav
/kaggle/input/hdramy-music-dataset/53_150.wav
/kaggle/input/hdramy-music-dataset/121_090.wav
/kaggle/input/hdramy-music-dataset/98_180.wav
/kaggle/input/hdramy-musi

In [2]:
import os
import librosa
import numpy as np
import pandas as pd

In [4]:
def is_good_sample(y, sr, min_duration=2.0, rms_threshold=0.01, zcr_threshold=(0.05, 0.3)): 
    # Check if the duration is above the minimum threshold
    duration = librosa.get_duration(y=y, sr=sr)
    if duration < min_duration:
        return False

    # Check if the RMS energy is above the threshold
    rms = np.mean(librosa.feature.rms(y=y))
    if rms < rms_threshold:
        return False

    # Check if the ZCR is within a reasonable range
    zcr = np.mean(librosa.feature.zero_crossing_rate(y))
    if not (zcr_threshold[0] <= zcr <= zcr_threshold[1]):
        return False

    return True

In [6]:
def extract_features(file_path, label):
    try:
        y, sr = librosa.load(file_path, sr=None)
        
        # Check if the sample is good
        if not is_good_sample(y, sr):
            print(f" ---Skipping {file_path} due to low quality.")
            return None
        
        # Initialize a dictionary to store features
        features = {}
        features['file_name'] = os.path.basename(file_path)
        features['label'] = label
        
        # Extract features and rename keys with a specific format
        features['mfcc_mean'] = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1).tolist()
        features['chroma_stft_mean'] = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1).tolist()
        features['spectral_contrast_mean'] = np.mean(librosa.feature.spectral_contrast(y=y, sr=sr), axis=1).tolist()
        features['spectral_centroid_mean'] = np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        features['spectral_rolloff_mean'] = np.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        features['zcr_mean'] = np.mean(librosa.feature.zero_crossing_rate(y))
        features['tempo'] = librosa.beat.beat_track(y=y, sr=sr)[0]
        features['tonnetz_mean'] = np.mean(librosa.feature.tonnetz(y=y, sr=sr), axis=1).tolist()
        features['rms_mean'] = np.mean(librosa.feature.rms(y=y))
        features['onset_mean'] = np.mean(librosa.onset.onset_strength(y=y, sr=sr))
        
        return features
    
    except Exception as e:
        # Handle exceptions by returning None or a dictionary with an error message
        print(f"Error processing {file_path}: {e}")
        return {'file_name': os.path.basename(file_path), 'label': label, 'error': str(e)}

def process_directory(directory_path, label, output_csv='extracted_features.csv'):
    features_list = []
    successful_extractions = 0
    total_samples = 0
    
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            if file.endswith(('.wav', '.mp3', '.flac')):
                total_samples += 1
                file_path = os.path.join(root, file)
                print('count : ',total_samples,'  filename:  ',file)
                features = extract_features(file_path, label)
                if features is not None and 'error' not in features:  # Only add features if the sample was good
                    features_list.append(features)
                    successful_extractions += 1
    
    # Convert the list of dictionaries to a DataFrame
    features_df = pd.DataFrame(features_list)
    
    # Only filter out rows with errors if the 'error' column exists
    if 'error' in features_df.columns:
        features_df = features_df[features_df['error'].isna()]
    
    # Expand lists into separate columns
    if not features_df.empty:
        mfcc_columns = pd.DataFrame(features_df['mfcc_mean'].tolist(), columns=[f'mfcc_{i}' for i in range(1, 14)])
        chroma_columns = pd.DataFrame(features_df['chroma_stft_mean'].tolist(), columns=[f'chroma_{i}' for i in range(1, 13)])
        spectral_contrast_columns = pd.DataFrame(features_df['spectral_contrast_mean'].tolist(), columns=[f'spectral_contrast_{i}' for i in range(1, 8)])
        tonnetz_columns = pd.DataFrame(features_df['tonnetz_mean'].tolist(), columns=[f'tonnetz_{i}' for i in range(1, 7)])
        
        # Concatenate everything back into a single DataFrame
        features_df = pd.concat([features_df.drop(columns=['mfcc_mean', 'chroma_stft_mean', 'spectral_contrast_mean', 'tonnetz_mean']),
                                 mfcc_columns, chroma_columns, spectral_contrast_columns, tonnetz_columns], axis=1)
    
    # Save to a CSV file
    features_df.to_csv(output_csv, index=False)
    
    print(f"Features extracted and saved to {output_csv}")
    print(f"Total samples processed: {total_samples}")
    print(f"Successful feature extractions: {successful_extractions}")
    
# Example usage:
directory_path = '/kaggle/input/hdramy-music-dataset'
label = 'hadrami'  # Example label, you can change this as needed
output_csv = '/kaggle/working/hadrami_style_features.csv'
process_directory(directory_path, label, output_csv)

count :  1   filename:   94_180.wav
count :  2   filename:   121_420.wav
count :  3   filename:   92_030.wav
count :  4   filename:   17_570.wav
 ---Skipping /kaggle/input/hdramy-music-dataset/17_570.wav due to low quality.
count :  5   filename:   27_1080.wav
 ---Skipping /kaggle/input/hdramy-music-dataset/27_1080.wav due to low quality.
count :  6   filename:   29_390.wav
count :  7   filename:   62_510.wav
count :  8   filename:   28_270.wav
count :  9   filename:   125_000.wav
count :  10   filename:   25_060.wav
count :  11   filename:   44_270.wav
count :  12   filename:   114_270.wav
count :  13   filename:   50_390.wav
count :  14   filename:   113_150.wav
count :  15   filename:   111_000.wav
count :  16   filename:   101_270.wav
count :  17   filename:   89_090.wav
count :  18   filename:   116_330.wav
count :  19   filename:   53_150.wav
count :  20   filename:   121_090.wav
count :  21   filename:   98_180.wav
count :  22   filename:   89_210.wav
count :  23   filename:   4